In [3]:
# pip install imageio-ffmpeg
!pip install moviepy
!pip install scikit-learn
!pip install ffmpeg-python


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached scikit_learn-1.5.0-cp310-cp310-win_amd64.whl (11.0 MB)
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
  Using cached scipy-1.13.1-cp310-cp310-win_amd64.whl (46.2 MB)



[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Import Libraries

In [4]:
import openai
import os
import re
import json
import string
from openai import OpenAI
from dotenv import load_dotenv
import subprocess

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

#### Extract Audio from Video for Transcription

In [8]:
from moviepy.editor import VideoFileClip

def extract_audio_with_moviepy(video_path, output_audio_path):
    try:
        video = VideoFileClip(video_path)
        video.audio.write_audiofile(output_audio_path)
        print(f'Audio extracted successfully: {output_audio_path}')
    except Exception as e:
        print(f'Error extracting audio: {e}')

# Define paths
video_file_path = './CaptionsGenerate.mp4'  # Replace with your video file path
audio_file_path = './Output/CaptionsGenerate.mp3'  # Extracted audio file path

# Extract audio from video
extract_audio_with_moviepy(video_file_path, audio_file_path)


MoviePy - Writing audio in ./Output/CaptionsGenerate.mp3


MoviePy - Done.
Audio extracted successfully: ./Output/CaptionsGenerate.mp3


#### Transcribing Audio to Text with Whisper API

In [14]:
import openai
import os
import json
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# Define the output directory for JSON and GIFs
OUTPUT_DIR = './Output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Function to transcribe audio using Whisper
def transcribe_audio_with_whisper(audio_path):
    print(f'Processing audio file: {audio_path}')
    with open(audio_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create(
            file=audio_file,
            model='whisper-1',
            response_format='verbose_json',
            timestamp_granularities=['word']
        )
        

    transcription_data = {                #word level
    "text": transcription.text,
    "task": transcription.task,
    "language": transcription.language,
    "duration": transcription.duration,
    "words": transcription.words
    }

    # Define the output filename
    output_filename = os.path.splitext(os.path.basename(audio_file_path))[0] + '.json'
    output_path = os.path.join(OUTPUT_DIR, output_filename)

    # Saving the translated text with timestamps to a JSON file
    with open(output_path, 'w') as f:
        json.dump(transcription_data, f, ensure_ascii=False, indent=4)

    # Print the path where the JSON file is saved
    print(f'Saved translation to {output_path}')

# Transcribe the extracted audio file
transcription_file_path = transcribe_audio_with_whisper(audio_file_path)


Processing audio file: ./Output/CaptionsGenerate.mp3
Saved translation to ./Output\CaptionsGenerateSeg.json


#### Extracting Expressions from the transcribed text

In [5]:
with open('Output/CaptionsGenerate.json') as f:
    data = json.load(f)

transcription_text = data['text']
transcription_text

"the test directly on this video. I'm gonna make some gifts. I love you. Grand Rising. Good vibes to all. Stay based. Thumbs up. Thumbs down. I'm okay. Happy Monday. Happy Tuesday. Happy Wednesday. Happy Thursday. Happy Friday. And a good New Year's Eve to everyone. And so basically it should choose where to section off each gift. It should be pretty obvious."

In [6]:
messages = [
    {
        "role": "system",
        "content": "You are an expert in analyzing and extracting notable moments and emotional expressions from transcriptions of video content. Your task is to meticulously analyze the provided transcription text to identify segments that capture distinct emotions, reactions, and significant expressions suitable for creating GIFs. You will output the results in a strict JSON format following a specified structure."
    },
    {
        "role": "user",
        "content": f'''
        Identify and extract segments from the following transcription text that are suitable for creating GIFs. Focus on moments that display clear emotions, reactions, and notable expressions, such as:

        - **Greetings and Farewells**: Friendly or enthusiastic opening and closing phrases like "Hello everyone", "Goodbye", "Grand Rising", "Happy Monday", etc.
        - **Sudden Emotions**: Exclamatory expressions or short, impactful phrases like "Wow!", "Oh no!", "Amazing!", "I love you", etc.
        - **Emotional Statements**: Sentences that convey a strong feeling like "I'm so happy", "This is frustrating", "I'm okay", etc.
        - **Reactions to Events**: Phrases showing a response to something happening like "That's incredible", "I can't believe it", etc.
        - **Laughter and Joy**: Any instances of laughter or expressions of joy like "Hahaha", "That's hilarious", "Woohoo!", etc.
        - **Excitement**: Enthusiastic or eager statements like "This is awesome", "I'm so excited", "Can't wait for this", etc.
        - **Surprises and Shock**: Reactions to unexpected events like "No way!", "What a surprise", etc.
        - **Emphatic Statements**: Strongly asserted phrases like "Absolutely!", "Definitely!", "Without a doubt", etc.
        - **Questioning or Curiosity**: Inquisitive expressions like "What's happening?", "Why is that?", "How does this work?", etc.
        - **Expressive Pauses**: Short, impactful sounds or words that convey a pause or hesitation like "Uh-oh", "Hmm", "Ahh", etc.
        - **Celebrations and Wishes**: Phrases used to celebrate or send good wishes like "Cheers!", "Good vibes to all", "Stay based", "And a good New Year's Eve to everyone", etc.
        - **Simple Gestures**: Indications of physical gestures like "Thumbs up", "Thumbs down".

        These moments are characterized by distinct phrasing, changes in tone, emotional emphasis, or visual cues that make them suitable for creating engaging and expressive GIFs.

        Below is the transcription text to analyze:

        "{transcription_text}"

        Return the identified segments in JSON format, including the type of emotion or expression and the text of each identified segment. Do not alter any text from the provided transcription.

        The output should strictly follow this format:
        {{
            "segments": [
                {{
                    "type": "Emotion or expression type",
                    "text": "Segment text"
                }},
                ...
            ]
        }}

        Ensure that:
        1. Each segment captures a distinct emotion or expression suitable for a GIF.
        2. The output adheres to the specified JSON format without any deviations.
        3. No text is changed from the original transcription.
        4. The type of emotion or expression is accurately identified or reasonably inferred based on the segment context.
        '''
    }
]

In [7]:
def extract_exp_segments(prompt, model):
    response = client.chat.completions.create(
        model=model,
        messages= prompt,
        response_format={"type":"json_object"},
        temperature=0,
    )
    return response

In [8]:
model = 'gpt-3.5-turbo'
segments = extract_exp_segments(messages, model)

In [9]:
res = segments.choices[0].message.content
jso = json.loads(res)
jso

{'segments': [{'type': 'Sudden Emotions', 'text': 'I love you.'},
  {'type': 'Greetings and Farewells', 'text': 'Grand Rising.'},
  {'type': 'Celebrations and Wishes', 'text': 'Good vibes to all.'},
  {'type': 'Celebrations and Wishes', 'text': 'Stay based.'},
  {'type': 'Simple Gestures', 'text': 'Thumbs up.'},
  {'type': 'Simple Gestures', 'text': 'Thumbs down.'},
  {'type': 'Emotional Statements', 'text': "I'm okay."},
  {'type': 'Greetings and Farewells', 'text': 'Happy Monday.'},
  {'type': 'Greetings and Farewells', 'text': 'Happy Tuesday.'},
  {'type': 'Greetings and Farewells', 'text': 'Happy Wednesday.'},
  {'type': 'Greetings and Farewells', 'text': 'Happy Thursday.'},
  {'type': 'Greetings and Farewells', 'text': 'Happy Friday.'},
  {'type': 'Celebrations and Wishes',
   'text': "And a good New Year's Eve to everyone."}]}

In [10]:
# Preprocess text: remove punctuation and lowercase
def preprocess_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    return text

# Tokenize text
def tokenize(text):
    return text.split()

#### Extracting timestamps of the expressions

In [11]:
# Extract timestamps for the expressions texts
def extract_timestamps(transcription, words, exp_data):
    text = preprocess_text(transcription)
    words_clean = [{'word': preprocess_text(w['word']), 'start': w['start'], 'end': w['end']} for w in words]
    transcription_tokens = [w['word'] for w in words_clean]
    
    results = []

    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer().fit(transcription_tokens)

    for exp in exp_data['segments']:
        advertisement_text = preprocess_text(exp['text'])
        advertisement_tokens = tokenize(advertisement_text)
        
        # Transform the expressions and transcription chunks into TF-IDF vectors
        ad_vector = vectorizer.transform([" ".join(advertisement_tokens)])
        transcription_vectors = vectorizer.transform([" ".join(transcription_tokens[i:i+len(advertisement_tokens)]) for i in range(len(transcription_tokens) - len(advertisement_tokens) + 1)])
        
        # Calculate cosine similarity
        cosine_similarities = cosine_similarity(ad_vector, transcription_vectors).flatten()
        
        # Find the best match index
        best_match_idx = np.argmax(cosine_similarities)
        
        # Get the start and end timestamps
        if best_match_idx != -1:
            start_timestamp = words_clean[best_match_idx]['start']
            end_timestamp = words_clean[best_match_idx + len(advertisement_tokens) - 1]['end']
            
            results.append({
                'type': exp['type'],
                'exp_text': exp['text'],
                'start_timestamp': start_timestamp,
                'end_timestamp': end_timestamp
            })
    return results

In [12]:
transcription_text = data['text']
word_list = data['words']
exp_segments_with_timestamps = extract_timestamps(transcription_text, word_list, jso)

# Print the results
for exp in exp_segments_with_timestamps:
    print(json.dumps(exp, indent=2))

{
  "type": "Sudden Emotions",
  "exp_text": "I love you.",
  "start_timestamp": 5.78000020980835,
  "end_timestamp": 9.0600004196167
}
{
  "type": "Greetings and Farewells",
  "exp_text": "Grand Rising.",
  "start_timestamp": 11.899999618530273,
  "end_timestamp": 13.319999694824219
}
{
  "type": "Celebrations and Wishes",
  "exp_text": "Good vibes to all.",
  "start_timestamp": 15.779999732971191,
  "end_timestamp": 16.8799991607666
}
{
  "type": "Celebrations and Wishes",
  "exp_text": "Stay based.",
  "start_timestamp": 18.959999084472656,
  "end_timestamp": 19.68000030517578
}
{
  "type": "Simple Gestures",
  "exp_text": "Thumbs up.",
  "start_timestamp": 22.360000610351562,
  "end_timestamp": 23.020000457763672
}
{
  "type": "Simple Gestures",
  "exp_text": "Thumbs down.",
  "start_timestamp": 24.719999313354492,
  "end_timestamp": 25.18000030517578
}
{
  "type": "Emotional Statements",
  "exp_text": "I'm okay.",
  "start_timestamp": 27.399999618530273,
  "end_timestamp": 29.3400

In [13]:
# Save the results to a JSON file
output_file = "./Output/CaptionOutput.json"
with open(output_file, 'w') as f:
    json.dump(exp_segments_with_timestamps, f, indent=2)

print(f"Results saved to {output_file}")

Results saved to ./Output/CaptionOutput.json


#### Converting Video to GIF

In [1]:
!ffmpeg -version

ffmpeg version 2024-06-16-git-fcf72966a5-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --enable-libfreetype --enable-libfribidi --enable-libharf

In [1]:
import os
import json
import ffmpeg

# Define paths
json_file_path = os.path.join("Output", "CaptionOutput.json")
video_file_path = "CaptionsGenerate.mp4"
output_dir = "GIF_Output"
gif_output_dir = "GIF_Created"
font_file_path = "Arial Bold.ttf"  # Full path to your font file

# Ensure the output directories exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

if not os.path.exists(gif_output_dir):
    os.makedirs(gif_output_dir)

# Read segment data from JSON file
with open(json_file_path, 'r') as file:
    segments = json.load(file)

# Function to trim video segments based on timestamps
def trim_video_segment(input_video, output_video, start_time, end_time):
    (
        ffmpeg
        .input(input_video, ss=start_time, to=end_time)
        .output(output_video, c="copy")
        .overwrite_output()
        .run(capture_stderr=True)
    )

# Function to overlay text on video and convert to GIF
def create_gif_with_text(segment, output_dir, gif_output_dir, font_file_path):
    start_time = segment["start_timestamp"]
    end_time = segment["end_timestamp"]
    text = segment["exp_text"]
    
    # Output file name and paths
    safe_text = text.replace(' ', '_').replace('\'', '')
    video_output_filename = f"{safe_text}.mp4"
    video_output_path = os.path.join(output_dir, video_output_filename)
    gif_output_filename = f"{safe_text}.gif"
    gif_output_path = os.path.join(gif_output_dir, gif_output_filename)
    
    # Trim the video segment
    trim_video_segment(video_file_path, video_output_path, start_time, end_time)
    
    # Overlay text on video and convert to GIF
    (
        ffmpeg
        .input(video_output_path)
        .output(gif_output_path, vf=f"drawtext=text='{text}':fontsize=100:fontcolor=red:fontfile={font_file_path}:x=(w-text_w)/2:y=h-th-10:bordercolor=white:borderw=7", vframes=25)
        .overwrite_output()
        .run(capture_stderr=True)
    )
    
    print(f"GIF created with caption '{text}' and saved as {gif_output_path}")

# Process each segment and create GIFs
for segment in segments:
    create_gif_with_text(segment, output_dir, gif_output_dir, font_file_path)

print("All videos converted to GIFs with captions successfully.")


GIF created with caption 'I love you.' and saved as GIF_Created\I_love_you..gif
GIF created with caption 'Grand Rising.' and saved as GIF_Created\Grand_Rising..gif
GIF created with caption 'Good vibes to all.' and saved as GIF_Created\Good_vibes_to_all..gif
GIF created with caption 'Stay based.' and saved as GIF_Created\Stay_based..gif
GIF created with caption 'Thumbs up.' and saved as GIF_Created\Thumbs_up..gif
GIF created with caption 'Thumbs down.' and saved as GIF_Created\Thumbs_down..gif
GIF created with caption 'I'm okay.' and saved as GIF_Created\Im_okay..gif
GIF created with caption 'Happy Monday.' and saved as GIF_Created\Happy_Monday..gif
GIF created with caption 'Happy Tuesday.' and saved as GIF_Created\Happy_Tuesday..gif
GIF created with caption 'Happy Wednesday.' and saved as GIF_Created\Happy_Wednesday..gif
GIF created with caption 'Happy Thursday.' and saved as GIF_Created\Happy_Thursday..gif
GIF created with caption 'Happy Friday.' and saved as GIF_Created\Happy_Friday.

        By Samit Dhawal 
        https://linkedin.com/in/samitdhawal/